In [0]:
# -*- coding: utf-8 -*-
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

In [0]:
import urllib.request

In [0]:
import time

mydataset = dataiku.Dataset("plane_tail_numbers")
df = mydataset.get_dataframe(limit=100000)


df = df.dropna()

df.index = range(len(df))

df["infos"] = ""
df["photo"] = ""
for i in df.index:
    try:
        url = "https://fr.flightaware.com/resources/registration/"+df.iloc[i].TailNum
        p = urllib.request.urlopen(url)
        page_content = soup(p.read())
        a = page_content.find("div", {"class":"medium-3 columns"}).text.replace("\n","")
        infos = " ".join(a.split("\t\t\t"))
        df.loc[i,("infos")] = infos
        #print i,")", df.ix[i].TailNum, infos
        time.sleep(0.5)
        df.loc[i,("photo")] = page_content.find("img", {"alt":"Thumbnail"})["src"]
    except:
        #print i,"ERROR ON", df.ix[i].TailNum
        time.sleep(0.01)


# Recipe outputs
aircraft_summary = dataiku.Dataset("aircraft_stats")
aircraft_summary.write_with_schema(df)